In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize

from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import random
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip

# load pickle module
import pickle
import networkx as nx

In [2]:
WN = np.loadtxt('W_avg.csv')
pop = np.loadtxt('pop_new.csv')

In [31]:
Cave = np.loadtxt('Cave_th50.csv')
# C = np.loadtxt('C.csv')
theta=1
Temp = np.zeros((3142,3142))
# w=zeros(size(Cave));
for j in range(3142):
    pop_j = pop[j]
    w_from_j = Cave[:,j]/pop_j
    Temp[:,j] = w_from_j

In [32]:
for j in range(3142):
    b = 1 - sum(Temp[:,j]) + Temp[j,j]
    if b < 0.2:
        Temp[j,j] = 0.2
    else:
        Temp[j,j] = b

In [29]:
for j in range(3142):
    Temp[:,j] = Temp[:,j]/np.sum(Temp[:,j])

In [33]:
Temp

array([[0.83241153, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94250428, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.94130276, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.97849303, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.97911595,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [15]:
WN

array([[0.83241153, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94250428, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.94130276, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.97849303, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.97911595,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [3]:
N = len(pop)

In [36]:
g_r = nx.from_numpy_array(WN)
g_r.remove_edges_from(nx.selfloop_edges(g_r))

In [35]:
nx.is_connected(g_r)

False

In [6]:
largest_component = max(nx.connected_components(g_r), key=len)

In [11]:
len(largest_component)

2546

In [13]:
1858 in list(largest_component)

True

In [38]:
subgraph = g_r.subgraph(largest_component)

In [39]:
nx.number_of_nodes(subgraph)

2546

In [40]:
nx.number_of_edges(subgraph)

7052